In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pptx import Presentation
from pptx.util import Inches


In [16]:
df = pd.read_csv("C:\\Users\\hanis\\OneDrive\\file1.csv", nrows=10000)
df_numeric = df.select_dtypes(include=[np.number])

X = df_numeric.drop(columns=["fire_weather_index"])
y = df_numeric["fire_weather_index"]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [19]:
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color="red", linestyle="--")
plt.xlabel("Predicted FWI")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.savefig("residual_plot.png")
plt.close()

In [20]:
alphas = np.logspace(-3, 3, 50)
coefs = []
for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(8,5))
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale("log")
plt.xlabel("Alpha")
plt.ylabel("Coefficient values")
plt.title("Ridge Coefficient Shrinkage")
plt.savefig("ridge_plot.png")
plt.close()

In [21]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

C:\Users\hanis\AppData\Roaming\Python\Python313\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


In [22]:
prs = Presentation("C:\\Users\\hanis\\Downloads\\Linear_Regression_FWI_Presentation.pptx")

In [23]:
slide_layout = prs.slide_layouts[5]  
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "Model Results & Plots (10k Sample)"

txBox = slide.shapes.add_textbox(Inches(0.5), Inches(1.2), Inches(4), Inches(1.5))
tf = txBox.text_frame
tf.text = f"Linear Regression Results:\n\nRMSE: {rmse:.2f}\nR²: {r2:.2f}"

slide.shapes.add_picture("residual_plot.png", Inches(0.5), Inches(3), height=Inches(2.5))
slide.shapes.add_picture("ridge_plot.png", Inches(5), Inches(3), height=Inches(2.5))

In [24]:
slide_layout = prs.slide_layouts[5]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "Multicollinearity (VIF)"

vif_text = "\n".join([f"{row['Feature']}: {row['VIF']:.2f}" for _, row in vif_data.head(10).iterrows()])
txBox = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(8), Inches(4))
txBox.text = "Top 10 Features with VIF:\n\n" + vif_text

In [25]:
prs.save("C:\\Users\\hanis\\Downloads\\Linear_Regression_FWI_Presentation.pptx")

print("✅ Final PPT saved as 'Linear_Regression_FWI_Final.pptx'")

✅ Final PPT saved as 'Linear_Regression_FWI_Final.pptx'
